# Create Solr Core

In [1]:
# Delete any existing core
!curl -X GET "http://localhost:8983/solr/admin/cores?action=UNLOAD&core=search_reddit&deleteInstanceDir=true&deleteDataDir=true"

{
  "responseHeader":{
    "status":0,
    "QTime":32
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    59  100    59    0     0    239      0 --:--:-- --:--:-- --:--:--   239


In [2]:
import shutil

# Create directory
shutil.copytree("C:\\solr-9.5.0-slim\\server\\solr\\configsets\\_default", "C:\\solr-9.5.0-slim\\server\\solr\\search_reddit")

'C:\\solr-9.5.0-slim\\server\\solr\\search_reddit'

In [3]:
# Create solr core with default settings
!curl -X GET "http://localhost:8983/solr/admin/cores?action=CREATE&name=search_reddit&instanceDir=search_reddit"

{
  "responseHeader":{
    "status":0,
    "QTime":18185
  },
  "core":"search_reddit"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:06 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:07 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:08 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:09 --:--:--     0
  0     0    0     0    0     0      0      0 --:--

# Solr recommends we use HTTP requests to edit schema. Because of data integrity issues

In [4]:
# Get current schema, should be default/reserved fields
!curl -X GET "http://localhost:8983/solr/search_reddit/schema/fields"

{
  "responseHeader":{
    "status":0,
    "QTime":0
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"id",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  }]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   574  100   574    0     0   2634      0 --:--:-- --:--:-- --:--:--  2645


# Add custom schema & index

https://solr.apache.org/guide/6_6/field-properties-by-use-case.html#field-properties-by-use-case
https://solr.apache.org/guide/6_6/field-type-definitions-and-properties.html#field-type-definitions-and-properties

## With only basic cleaning done beforehand, the rest have to be done in Solr. Therefore we define a custom field type for text

- https://solr.apache.org/guide/6_6/filter-descriptions.html

```
  <fieldType name="text_reddit" class="solr.TextField" positionIncrementGap="100" multiValued="true">
    <analyzer type="index">
      <tokenizer class="solr.StandardTokenizerFactory"/>
      <filter class="solr.LowerCaseFilterFactory"/>
      <filter class="solr.StopFilterFactory" ignoreCase="true" words="stopwords.txt"/>
      <filter class="solr.SnowballPorterFilterFactory" language="English"/>
      <filter class="solr.NGramFilterFactory" minGramSize="1" maxGramSize="10"/>
    </analyzer>
    <analyzer type="query">
      <tokenizer class="solr.StandardTokenizerFactory"/>
      <filter class="solr.LowerCaseFilterFactory"/>
      <filter class="solr.StopFilterFactory" ignoreCase="true" words="stopwords.txt"/>
      <filter class="solr.SnowballPorterFilterFactory" language="English"/>
      <filter class="solr.NGramFilterFactory" minGramSize="1" maxGramSize="10"/>
      <filter class="solr.SynonymGraphFilterFactory" ignoreCase="true" synonyms="synonyms.txt" expand="true"/>
    </analyzer>
```

In [5]:
# Add specific filtering/analysis operations
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field-type":{"name":"text_reddit","class":"solr.TextField","positionIncrementGap":"100","multiValued":true,"indexAnalyzer":{"tokenizer":{"class":"solr.StandardTokenizerFactory"},"filters":[{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.SnowballPorterFilterFactory","language":"English"},{"class":"solr.NGramFilterFactory","minGramSize":1,"maxGramSize":10}]},"queryAnalyzer":{"tokenizer":{"class":"solr.StandardTokenizerFactory"},"filters":[{"class":"solr.LowerCaseFilterFactory"},{"class":"solr.StopFilterFactory","ignoreCase":true,"words":"stopwords.txt"},{"class":"solr.SnowballPorterFilterFactory","language":"English"},{"class":"solr.NGramFilterFactory","minGramSize":1,"maxGramSize":10},{"class":"solr.SynonymGraphFilterFactory","synonyms":"synonyms.txt","expand":true,"ignoreCase":true}]}}}" http://localhost:8983/api/cores/search_reddit/schema

{
  "responseHeader":{
    "status":0,
    "QTime":18211
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   779    0     0  100   779      0    635  0:00:01  0:00:01 --:--:--   635
100   779    0     0  100   779      0    347  0:00:02  0:00:02 --:--:--   347
100   779    0     0  100   779      0    240  0:00:03  0:00:03 --:--:--   240
100   779    0     0  100   779      0    183  0:00:04  0:00:04 --:--:--   183
100   779    0     0  100   779      0    148  0:00:05  0:00:05 --:--:--   148
100   779    0     0  100   779      0    124  0:00:06  0:00:06 --:--:--     0
100   779    0     0  100   779      0    107  0:00:07  0:00:07 --:--:--     0
100   779    0     0  100   779      0     94  0:00:08  0:00:08 --:--:--     0
100   779    0     0  100   779      0     83  0:00:09  0:00:09 --:--:--     0
100   779    0     0  100   779      0     75  0:00

## Field types explained
- indexed -> If true, the value of the field can be used in queries to retrieve matching documents.
- stored -> If true, the actual value of the field can be retrieved by queries.

Our search engine primarily searches by comment & post texts

```
<field name="_nest_path_" type="_nest_path_"/>
<field name="_root_" type="string" indexed="true" stored="false" docValues="false"/>
<field name="_text_" type="text_general" multiValued="true" indexed="true" stored="false"/>
<field name="_version_" type="plong" indexed="false" stored="false"/>
<field name="author" type="text_general"/>
<field name="created_utc" type="pdates"/>
<field name="edited" type="pdates"/>
<field name="id" type="string" multiValued="false" indexed="true" required="true" stored="true"/>
<field name="num_comments" type="pdoubles"/>
<field name="permalink" type="text_general"/>
<field name="post_id" type="text_general"/>
<field name="score" type="plongs"/>
<field name="subreddit_name" type="text_general"/>
<field name="text" type="text_general"/>
<field name="type" type="text_general"/>
<field name="upvote_ratio" type="pdoubles"/>
<field name="url" type="text_general"/>
```

In [6]:
# Author, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"author","type":"string","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# Text, searched and retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"text","type":"text_reddit","stored":true,"indexed":true,"required":true}}" http://localhost:8983/api/cores/search_reddit/schema

# created_utc, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"created_utc","type":"pdates","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# edited, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"edited","type":"pdates","stored":true,"indexed":true}}" http://localhost:8983/api/cores/search_reddit/schema

# id, retrieved, unique key
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"id","type":"string","stored":true,"indexed":true,"multiValued":false,"required":true}}" http://localhost:8983/api/cores/search_reddit/schema

# num_comments, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"num_comments","type":"pdoubles","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# permalink, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"permalink","type":"string","stored":true,"multiValued":false}}" http://localhost:8983/api/cores/search_reddit/schema

# score, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"score","type":"plong","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# subreddit_name, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"subreddit_name","type":"string","stored":true,"multiValued":true}}" http://localhost:8983/api/cores/search_reddit/schema

# upvote_ratio, retrieved, sorted by
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"upvote_ratio","type":"pdoubles","stored":true,"indexed":true, "multiValued":false, "omitNorms":true,"docValues":true}}" http://localhost:8983/api/cores/search_reddit/schema

# url, retrieved
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"url","type":"string","stored":true,"multiValued":false}}" http://localhost:8983/api/cores/search_reddit/schema

# type, retrieved, have duplicates
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"type","type":"string","stored":true,"multiValued":true}}" http://localhost:8983/api/cores/search_reddit/schema

# post_id, retrieved, have duplicates
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"post_id","type":"string","stored":true,"multiValued":true}}" http://localhost:8983/api/cores/search_reddit/schema

{
  "responseHeader":{
    "status":0,
    "QTime":18188
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   112    0     0  100   112      0     92  0:00:01  0:00:01 --:--:--    92
100   112    0     0  100   112      0     50  0:00:02  0:00:02 --:--:--    50
100   112    0     0  100   112      0     34  0:00:03  0:00:03 --:--:--    34
100   112    0     0  100   112      0     26  0:00:04  0:00:04 --:--:--    26
100   112    0     0  100   112      0     21  0:00:05  0:00:05 --:--:--    22
100   112    0     0  100   112      0     17  0:00:06  0:00:06 --:--:--     0
100   112    0     0  100   112      0     15  0:00:07  0:00:07 --:--:--     0
100   112    0     0  100   112      0     13  0:00:08  0:00:08 --:--:--     0
100   112    0     0  100   112      0     12  0:00

{
  "responseHeader":{
    "status":0,
    "QTime":18193
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    79    0     0  100    79      0     65  0:00:01  0:00:01 --:--:--    65
100    79    0     0  100    79      0     35  0:00:02  0:00:02 --:--:--    35
100    79    0     0  100    79      0     24  0:00:03  0:00:03 --:--:--    24
100    79    0     0  100    79      0     18  0:00:04  0:00:04 --:--:--    18
100    79    0     0  100    79      0     15  0:00:05  0:00:05 --:--:--    15
100    79    0     0  100    79      0     12  0:00:06  0:00:06 --:--:--     0
100    79    0     0  100    79      0     10  0:00:07  0:00:07 --:--:--     0
100    79    0     0  100    79      0      9  0:00:08  0:00:08 --:--:--     0
100    79    0     0  100    79      0      8  0:00:09  0:00:09 --:--:--     0
100    79    0     0  100    79      0      7  0:00

{
  "responseHeader":{
    "status":0,
    "QTime":18220
  }
}


...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    64    0     0  100    64      0     52  0:00:01  0:00:01 --:--:--    52
100    64    0     0  100    64      0     28  0:00:02  0:00:02 --:--:--    28
100    64    0     0  100    64      0     19  0:00:03  0:00:03 --:--:--    19
100    64    0     0  100    64      0     15  0:00:04  0:00:04 --:--:--    15
100    64    0     0  100    64      0     12  0:00:05  0:00:05 --:--:--    12
100    64    0     0  100    64      0     10  0:00:06  0:00:06 --:--:--     0
100    64    0     0  100    64      0      8  0:00:08  0:00:07  0:00:01     0
100    64    0     0  100    64      0      7  0:00:09  0:00:08  0:00:01     0
100    64    0     0  100    64      0      6  0:00:10  0:00:09  0:00:01     0
100    64    0     0  100    64      0      6  0:00

{
  "responseHeader":{
    "status":0,
    "QTime":18193
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    64    0     0  100    64      0     52  0:00:01  0:00:01 --:--:--    52
100    64    0     0  100    64      0     28  0:00:02  0:00:02 --:--:--    28
100    64    0     0  100    64      0     19  0:00:03  0:00:03 --:--:--    19
100    64    0     0  100    64      0     15  0:00:04  0:00:04 --:--:--    15
100    64    0     0  100    64      0     12  0:00:05  0:00:05 --:--:--    12
100    64    0     0  100    64      0     10  0:00:06  0:00:06 --:--:--     0
100    64    0     0  100    64      0      8  0:00:08  0:00:07  0:00:01     0
100    64    0     0  100    64      0      7  0:00:09  0:00:08  0:00:01     0
100    64    0     0  100    64      0      6  0:00:10  0:00:09  0:00:01     0
100    64    0     0  100    64      0      6  0:00

{
  "responseHeader":{
    "status":0,
    "QTime":18224
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    67    0     0  100    67      0     55  0:00:01  0:00:01 --:--:--    55
100    67    0     0  100    67      0     30  0:00:02  0:00:02 --:--:--    30
100    67    0     0  100    67      0     20  0:00:03  0:00:03 --:--:--    20
100    67    0     0  100    67      0     15  0:00:04  0:00:04 --:--:--    15
100    67    0     0  100    67      0     12  0:00:05  0:00:05 --:--:--    12
100    67    0     0  100    67      0     10  0:00:06  0:00:06 --:--:--     0
100    67    0     0  100    67      0      9  0:00:07  0:00:07 --:--:--     0
100    67    0     0  100    67      0      8  0:00:08  0:00:08 --:--:--     0
100    67    0     0  100    67      0      7  0:00:09  0:00:09 --:--:--     0
100    67    0     0  100    67      0      6  0:00

In [7]:
# Get current schema, should be default/reserved fields
!curl -X GET "http://localhost:8983/solr/search_reddit/schema/fields"

{
  "responseHeader":{
    "status":0,
    "QTime":0
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"author",
    "type":"string",
    "omitNorms":true,
    "docValues":true,
    "multiValued":false,
    "indexed":true,
    "stored":true
  },{
    "name":"created_utc",
    "type":"pdates",
    "omitNorms":true,
    "docValues":true,
    "multiValued":false,
    "indexed":true,
    "stored":true
  },{
    "name":"edited",
    "type":"pdates",
    "indexed":true,
    "stored":true
  },{
    "name":"id",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  },{
    "name":"num_comments"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2023  100  2023    0     0   9522      0 --:--:-- --:--:-- --:--:--  9542


# Ingest data

In [8]:
# add document to solr
!curl "http://localhost:8983/solr/search_reddit/update?commit=true" --data-binary @data/cleaned_combined_data.csv -H "Content-type:application/csv"

{
  "responseHeader":{
    "status":0,
    "QTime":4982
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 37 13.8M    0     0   37 5312k      0  5636k  0:00:02 --:--:--  0:00:02 5633k
 55 13.8M    0     0   55 7808k      0  4598k  0:00:03  0:00:01  0:00:02 4598k
 75 13.8M    0     0   75 10.4M      0  4269k  0:00:03  0:00:02  0:00:01 4268k
 93 13.8M    0     0   93 12.8M      0  3917k  0:00:03  0:00:03 --:--:-- 3916k
100 13.8M    0     0  100 13.8M      0  3242k  0:00:04  0:00:04 --:--:-- 3242k
100 13.8M  100    61  100 13.8M     11  2732k  0:00:05  0:00:05 --:--:-- 2088k


# Test query data

- https://solr.apache.org/guide/6_6/the-standard-query-parser.html

## Select with keyword search

In [9]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=text:congrats*&rows=10"

{
  "responseHeader":{
    "status":0,
    "QTime":1,
    "params":{
      "q":"text:congrats*",
      "rows":"10"
    }
  },
  "response":{
    "numFound":0,
    "start":0,
    "numFoundExact":true,
    "docs":[ ]
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   244  100   220  100    24   1015    110 --:--:-- --:--:-- --:--:--  1124


## Search with timeframe range filter

In [10]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]&rows=10"

{
  "responseHeader":{
    "status":0,
    "QTime":1,
    "params":{
      "q":"created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]",
      "rows":"10"
    }
  },
  "response":{
    "numFound":1935,
    "start":0,
    "numFoundExact":true,
    "docs":[{
      "author":"billbucket",
      "text":["Tesla's Summon feature was very useful today..."],
      "created_utc":"2018-01-25T07:13:35Z",
      "id":"7srdiw",
      "num_comments":1050.0,
      "permalink":"/r/teslamotors/comments/7srdiw/teslas_summon_feature_was_very_useful_today/",
      "score":49182,
      "subreddit_name":["teslamotors"],
      "upvote_ratio":0.9,
      "url":"https://gfycat.com/ReliableSecretJunebug",
      "type":["post"],
      "_version_":1794257300475609103
    },{
      "author":"TheMightyCraken",
      "text":["Elon Musk confirms secret Tesla â€˜Carless Driverâ€™ project"],
      "created_utc":"2018-01-30T05:20:51Z",
      "id":"7tvnx6",
      "num_comments":394.0,
      "permalink":"/r/teslamotors/c

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5243  100  5175  100    68  25114    330 --:--:-- --:--:-- --:--:-- 25451


## Combination of conditions

In [11]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=text:congrats* AND created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]&rows=10"

{
  "responseHeader":{
    "status":0,
    "QTime":1,
    "params":{
      "q":"text:congrats* AND created_utc:[2017-01-01T00:00:00Z TO 2018-01-31T23:59:59Z]",
      "rows":"10"
    }
  },
  "response":{
    "numFound":0,
    "start":0,
    "numFoundExact":true,
    "docs":[ ]
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   370  100   283  100    87   1295    398 --:--:-- --:--:-- --:--:--  1697
